In [26]:
from facenet_pytorch import MTCNN
import cv2
from PIL import ImageDraw
import numpy as np
import torch
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import time
import glob
from moviepy.editor import *
from PIL import Image as Img
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [27]:
# Create face detector
mtcnn = MTCNN(margin=20, keep_all=True, post_process=False, device=device)
print(device)

cuda


In [28]:
# # Load a video
# v_cap = cv2.VideoCapture('./input.mp4')
# v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))

# destination_name = lambda x: os.path.join('frames', x)

# # Loop through video
# batch_size = 32
# frames = []
# boxes = []
# landmarks = []
# all_frames = []
# probs = []
# for i in tqdm(range(v_len)):
    
#     # Load frame
#     success, frame = v_cap.read()
#     if not success:
#         continue
#     frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     frame = Img.fromarray(frame)
#     frame.save(destination_name('img_{}.jpg'.format(i)))
    
#     # When batch is full, detect faces and reset batch list
#     if len(frames) >= batch_size:
#         batch_boxes, p, batch_landmarks = mtcnn.detect(frames, landmarks=True)
#         boxes.extend(batch_boxes)
#         probs.extend(p)
#         frames = []

In [29]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams['figure.figsize'] = (8, 8)
mpl.rcParams['axes.grid'] = False

In [30]:
pretrained_model = tf.keras.applications.MobileNetV2(include_top=True,
                                                     weights='imagenet')
pretrained_model.trainable = False

# ImageNet labels
decode_predictions = tf.keras.applications.mobilenet_v2.decode_predictions

In [31]:
# Helper function to preprocess the image so that it can be inputted in MobileNetV2
def preprocess(image):
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, (224, 224))
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    image = image[None, ...]
    return image

# Helper function to extract labels from probability vector
def get_imagenet_label(probs):
  return decode_predictions(probs, top=1)[0][0]

In [32]:
loss_object = tf.keras.losses.CategoricalCrossentropy()

def create_adversarial_pattern(input_image, input_label):
  with tf.GradientTape() as tape:
    tape.watch(input_image)
    prediction = pretrained_model(input_image)
    loss = loss_object(input_label, prediction)

  # Get the gradients of the loss w.r.t to the input image.
  gradient = tape.gradient(loss, input_image)
  # Get the sign of the gradients to create the perturbation
  signed_grad = tf.sign(gradient)
  return signed_grad

In [15]:
image_paths = glob.glob(os.path.join('frames', "*"))


In [16]:
for image_path in tqdm(image_paths):
    ix = image_path.split('.')[0].split('_')[1]
    image_raw = tf.io.read_file(image_path)
    image = tf.image.decode_image(image_raw)

    image = preprocess(image)
    image_probs = pretrained_model.predict(image)
    labrador_retriever_index = 208
    label = tf.one_hot(labrador_retriever_index, image_probs.shape[-1])
    label = tf.reshape(label, (1, image_probs.shape[-1]))

    perturbations = create_adversarial_pattern(image, label)
    adv_x = image + 0.01*perturbations
    tf.keras.preprocessing.image.save_img('processed/' + ix.zfill(8) + '_.png',adv_x[0])

  0%|          | 0/1294 [00:00<?, ?it/s]

In [19]:
image_paths = glob.glob(os.path.join('processed', "*"))
image_paths = sorted(image_paths)

In [25]:
writer = cv2.VideoWriter('processed_temp.mp4', cv2.VideoWriter_fourcc(*'MJPG'), 25, (1280,720))
for img_path in tqdm(image_paths):
    img = cv2.imread(img_path)
    img = Img.fromarray(img)
    img = img.resize((1280, 720))
    img = np.array(img)
    writer.write(img)
writer.release()

  0%|          | 0/1295 [00:00<?, ?it/s]

In [38]:
frame = Img.fromarray(img)
frame.resize((1280, 720))
mtcnn.detect(frame)

(None, array([None], dtype=object))

In [39]:
frame.show()

None
